<a href="https://colab.research.google.com/github/Is7ac/ML/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
dataset_dir = "/content/drive/MyDrive/chest_xray/train"

In [3]:
# Step 2: Image Preprocessing

# Define a function to preprocess images
def preprocess_image(image_path, target_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image_vector = image.flatten()  # Flatten the image
    return image_vector

In [9]:
# Define target image size
target_size = (128, 128)

# Load images and labels
images = []
labels = []

for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image_vector = preprocess_image(image_path, target_size)
        images.append(image_vector)
        labels.append(class_name)  # Assuming folder names are class labels

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)


In [10]:
# Step 3: Classification - Fully Connected Neural Network

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [11]:
# Define the neural network manually
class FullyConnectedNN(tf.keras.Model):
    def __init__(self):
        super(FullyConnectedNN, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(np.prod(target_size),))
        self.fc1 = tf.keras.layers.Dense(128, activation='relu')
        self.fc2 = tf.keras.layers.Dense(64, activation='relu')
        self.output_layer = tf.keras.layers.Dense(2, activation='softmax')  # Assuming binary classification (2 classes)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.output_layer(x)


In [14]:
# Create an instance of the model
model = FullyConnectedNN()

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Define batch size and number of epochs
batch_size = 32
epochs = 10

# Convert string labels to integer labels
label_map = {label: idx for idx, label in enumerate(np.unique(labels))}
y_train = np.array([label_map[label] for label in y_train])
y_test = np.array([label_map[label] for label in y_test])

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)

Epoch 1/10
105/105 [==============================] - 15s 128ms/step - loss: 208.0339 - accuracy: 0.8343 - val_loss: 245.2862 - val_accuracy: 0.4599
Epoch 2/10
105/105 [==============================] - 11s 103ms/step - loss: 92.1698 - accuracy: 0.8592 - val_loss: 35.9933 - val_accuracy: 0.9138
Epoch 3/10
105/105 [==============================] - 12s 110ms/step - loss: 31.1563 - accuracy: 0.9170 - val_loss: 14.1996 - val_accuracy: 0.9521
Epoch 4/10
105/105 [==============================] - 12s 111ms/step - loss: 24.1085 - accuracy: 0.9263 - val_loss: 23.0227 - val_accuracy: 0.9234
Epoch 5/10
105/105 [==============================] - 12s 112ms/step - loss: 45.8121 - accuracy: 0.9017 - val_loss: 983.5479 - val_accuracy: 0.7437
Epoch 6/10
105/105 [==============================] - 11s 109ms/step - loss: 40.8400 - accuracy: 0.9308 - val_loss: 11.8164 - val_accuracy: 0.9485
Epoch 7/10
105/105 [==============================] - 11s 102ms/step - loss: 22.2689 - accuracy: 0.9179 - val_loss: